In [40]:
#basic imports
%matplotlib inline

import matplotlib
import matplotlib.pyplot as plt

import pymongo as pm
import numpy as np
import h5py
import scipy as sp
import scipy.stats as stats
import os

#import PLS Regression routine from scikit
from sklearn.cross_decomposition import PLSRegression

In [5]:
# read in the alexnet features
h5f = h5py.File('alexnet/alexnet_conv3_features.h5','r')
train_features = h5f['train'][:]
val_features = h5f['val'][:]
h5f.close()

In [7]:
train_features.shape

(1750, 14, 14, 384)

In [9]:
# let's bring in the brain data

#read in the mat files
with h5py.File('data/EstimatedResponses.mat','r') as fmri_dataset:
    train_S1 = fmri_dataset['dataTrnS1'][:]
    test_S1 = fmri_dataset['dataValS1'][:]
    roi_S1 = fmri_dataset['roiS1'][:]
    
    train_S2 = fmri_dataset['dataTrnS2'][:]
    test_S2 = fmri_dataset['dataValS2'][:]
    roi_S2 = fmri_dataset['roiS2'][:]
    
    unique_ROIs = np.unique((fmri_dataset['roiS1'])) 

In [17]:
#organize subject 1
S1_train_data_by_ROI = {c: train_S1[:,roi_S1[0,:] == c] 
                       for c in unique_ROIs}
S1_test_data_by_ROI = {c: test_S1[:,roi_S1[0,:] == c] 
                       for c in unique_ROIs}
#organize subject 2
S2_train_data_by_ROI = {c: train_S2[:,roi_S2[0,:] == c] 
                       for c in unique_ROIs}
S2_test_data_by_ROI = {c: test_S2[:,roi_S2[0,:] == c] 
                       for c in unique_ROIs}

In [18]:
S1_train_data_by_ROI.keys()
# remember the one we care about is index 6 for V4

[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0]

In [20]:
#okay how many voxels are we working with
print(S1_train_data_by_ROI[6.0].shape)
print(S2_train_data_by_ROI[6.0].shape)

(1750, 1550)
(1750, 1022)


In [143]:
#let's get rid of any voxels that have NaNs for any training images

#Subject 1
S1_V4 = S1_train_data_by_ROI[6.0].T
x=S1_V4[~np.isnan(S1_V4).any(axis=1)]
S1_V4_train = x.T
print(S1_V4_train.shape)

S1_V4_val = S1_test_data_by_ROI[6.0].T
x=S1_V4_val[~np.isnan(S1_V4).any(axis=1)]
S1_V4_test = x.T
print(S1_V4_test.shape)

sum(sum(np.isnan(S1_V4_test))) #make sure we don't have any NaNs left in the test data

#Subject 2
S2_V4 = S2_train_data_by_ROI[6.0].T
x=S2_V4[~np.isnan(S2_V4).any(axis=1)]
S2_V4_train = x.T
print(S2_V4_train.shape)

S2_V4_val = S2_test_data_by_ROI[6.0].T
x=S2_V4_val[~np.isnan(S2_V4).any(axis=1)]
S2_V4_test = x.T
print(S2_V4_test.shape)

sum(sum(np.isnan(S2_V4_test))) #make sure we don't have any NaNs left in the test data

(1750, 1535)
(120, 1535)
(1750, 1022)
(120, 1022)


0

In [83]:
def rsquared(v1, v2):
    w, b, r, p, ser = stats.linregress(v1, v2)
    return r**2

In [84]:
def evaluate_regression_results(predicted, actual):
    """computing various useful metrics for regression results
    """
    result = {}
    if actual.ndim > 1: #this is triggered if the prediction is of multiple outputs at once
        result['pearson_array'] = np.array([stats.pearsonr(p, a)[0] for p, a in zip(predicted.T, actual.T)])
        result['spearman_array'] = np.array([stats.spearmanr(p, a)[0] for p, a in zip(predicted.T, actual.T)])
        result['rsquared_array'] = np.array([rsquared(p, a) for p, a in zip(predicted.T, actual.T)])
        result['pearson'] = np.median(result['pearson_array'])
        result['spearman'] = np.median(result['spearman_array'])
        result['rsquared'] = np.median(result['rsquared_array'])
    else:
        result['pearson'] = stats.pearsonr(predicted, actual)[0]
        result['spearman'] = stats.spearmanr(predicted, actual)[0]
        result['rsquared'] = rsquared(predicted, actual)
    return result


In [125]:
pls25 = PLSRegression(n_components=50)#, scale=False)

In [126]:
pls25

PLSRegression(copy=True, max_iter=500, n_components=50, scale=True, tol=1e-06)

In [127]:
#flatten the conv features
shp = train_features.shape
train_feats = train_features.reshape((shp[0], np.prod(shp[1:])))
print(train_feats.shape)

shp = val_features.shape
val_feats = val_features.reshape((shp[0], np.prod(shp[1:])))
print(val_feats.shape)

(1750, 75264)
(120, 75264)


In [128]:
pls25.fit(train_feats, S1_V4_train)

PLSRegression(copy=True, max_iter=500, n_components=50, scale=True, tol=1e-06)

In [129]:
S1_pred_train_vox = pls25.predict(train_feats)

In [130]:
train_results = evaluate_regression_results(S1_pred_train_vox,S1_V4_train)

In [131]:
train_results['rsquared_array'].shape

(1535,)

In [132]:
train_results

{'pearson': 0.3920543583208737,
 'pearson_array': array([0.52400149, 0.23512401, 0.19664139, ..., 0.37302986, 0.25043704,
        0.35441066]),
 'rsquared': 0.1537066198783919,
 'rsquared_array': array([0.27457756, 0.0552833 , 0.03866784, ..., 0.13915127, 0.06271871,
        0.12560692]),
 'spearman': 0.369251402671012,
 'spearman_array': array([0.51420718, 0.2243858 , 0.17645073, ..., 0.35844459, 0.24480546,
        0.31393106])}